In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import random

# 제품 데이터 저장 리스트
products_data = []

# Selenium 드라이버 설정
def setup_driver():
    chrome_options = webdriver.ChromeOptions()
    # chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    # Selenium 탐지 우회
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

# 제품 정보 추출 함수
def extract_product_info(product, driver):
    try:
        if product.get("id", "").startswith("adPointProductItem") or "adSmartAreaTop" in product.get("id", ""):
            return False

        photo_elem = product.select_one(".thumb_image img")
        if photo_elem:
            photo = photo_elem.get("data-original") or photo_elem.get("src") or "N/A"
            if "noImg_130.gif" in photo:
                photo = "N/A"
        else:
            photo = "N/A"

        name_elem = product.select_one(".prod_name a")
        name = name_elem.text.strip() if name_elem else "Unknown"
        detail_url = name_elem["href"] if name_elem and "href" in name_elem.attrs else None

        # 상세 페이지에서 spec 수집
        spec_dict = {}
        if detail_url:
            # 현재 탭의 핸들 저장
            main_window = driver.current_window_handle

            # 새 탭 열기 (JavaScript 실행)
            driver.execute_script("window.open('', '_blank');")

            # 새 탭으로 전환
            driver.switch_to.window(driver.window_handles[-1])

            # 새 탭에서 상세 페이지 열기
            driver.get(detail_url)
            try:
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_spec table"))
                )
            except:
                print(f"❗️스펙 테이블 로딩 실패 (대기 시간 초과): {name}")

            detail_soup = BeautifulSoup(driver.page_source, "html.parser")

            spec_table = detail_soup.select_one(".prod_spec .spec_tbl tbody") or detail_soup.select_one(".prod_spec table")
            if spec_table:
                for row in spec_table.find_all("tr"):
                    keys = row.find_all("th", class_="tit")
                    values = row.find_all("td", class_="dsc")
                    for key_elem, value_elem in zip(keys, values):
                        key = key_elem.get_text(strip=True)
                        value = value_elem.get_text(strip=True)
                        spec_dict[key] = value

            # 탭 닫기
            driver.close()

            # 원래 탭으로 돌아가기
            driver.switch_to.window(main_window)

        price_elem = product.select_one(".price_sect a")
        price = price_elem.text.strip().replace(",", "") if price_elem else "N/A"

        rating_elem = product.select_one(".star-single .text__score") or product.select_one(".point_num")
        rating = rating_elem.text.strip() if rating_elem else "N/A"

        product_info = {
            "url": photo,
            "Name": name,
            "Price": price,
            "Rating": rating,
            **spec_dict
        }
        products_data.append(product_info)
        print(f"수집: {name}, 사진: {photo}, 가격: {price}, 평점: {rating}")
        return True

    except Exception as e:
        print(f"제품 정보 수집 오류: {e}")

        # 혹시 새 탭이 열려있는 상태라면 닫고 메인 탭으로 돌아가기
        try:
            if len(driver.window_handles) > 1:
                driver.close()
                driver.switch_to.window(driver.window_handles[0])
        except:
            pass

        return False

# 페이지 스크레이핑 함수
def scrape_page(driver, page_num):
    try:
        # 현재 페이지의 제품 정보 수집
        soup = BeautifulSoup(driver.page_source, "html.parser")
        products = soup.select(".prod_item.prod_layer")
        print(f"페이지 {page_num}에서 찾은 제품 수: {len(products)}")

        if not products:
            print("더 이상 제품이 없습니다.")
            return False

        for product in products:
            if product.find_parent(id="adPointArea") or product.find_parent(id="adSmartAreaTop"):
                continue
            extract_product_info(product, driver)

        return True
    except Exception as e:
        print(f"페이지 수집 오류 (페이지 {page_num}): {e}")
        return False

# 크롤링 함수
def crawl_danawa_cpu_to_dataframe(max_pages=None):
    driver = setup_driver()
    try:
        base_url = "https://prod.danawa.com/list/?cate=112763"
        driver.get(base_url)
        time.sleep(3)  # 초기 페이지 로딩 대기

        page_num = 1
        while True:
            if max_pages and page_num > max_pages:
                break

            print(f"페이지 {page_num} 수집 중...")

            # 현재 페이지 수집
            scrape_page(driver, page_num)

            if max_pages and page_num >= max_pages:
                break

            # 다음 페이지로 이동
            try:
                # 페이지 이동 방식 결정
                if page_num % 10 == 0:  # 10, 20, 30 등 10의 배수 페이지일 경우
                    # 다음 페이지 그룹 버튼 클릭
                    next_page_button = WebDriverWait(driver, 5).until(
                        EC.element_to_be_clickable((By.CSS_SELECTOR, "a.edge_nav.nav_next"))
                    )
                    next_page_button.click()
                else:
                    # 일반 페이지 이동
                    driver.execute_script(f"movePage({page_num + 1}); return false;")

                # 페이지 로딩 대기
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, ".prod_item.prod_layer"))
                )
                time.sleep(2)  # 추가 대기 시간

                page_num += 1
            except Exception as e:
                print(f"다음 페이지로 이동 실패: {e}")
                break

        df = pd.DataFrame(products_data)
        print(f"총 {len(products_data)}개 CPU 제품 수집 완료")
        return df

    finally:
        driver.quit()
        print("Selenium 드라이버 종료")

# 실행
max_pages = 18  # 테스트를 위해 작은 값으로 설정
df = crawl_danawa_cpu_to_dataframe(max_pages=max_pages)
print("\n수집된 데이터프레임 미리보기:")
print(df.head())
df.to_csv("danawa_hdd_products.csv", index=False, encoding="utf-8-sig")
print("데이터가 'danawa_hdd_products.csv' 파일로 저장되었습니다.")

페이지 1 수집 중...
페이지 1에서 찾은 제품 수: 30
수집: Seagate BarraCuda 5400/256M, 사진: //img.danawa.com/prod_img/500000/992/764/img/5764992_1.jpg?shrink=130:130&_v=20220629141336, 가격: 199380원, 평점: 4.7
수집: Seagate BarraCuda 7200/256M, 사진: //img.danawa.com/prod_img/500000/078/545/img/6545078_1.jpg?shrink=130:130&_v=20220629141246, 가격: 92190원, 평점: 4.7
수집: Western Digital WD Blue 5640/256M, 사진: //img.danawa.com/prod_img/500000/460/286/img/42286460_1.jpg?shrink=130:130&_v=20250331161408, 가격: 209900원, 평점: 4.7
수집: Western Digital WD Blue 7200/256M, 사진: //img.danawa.com/prod_img/500000/751/522/img/13522751_1.jpg?shrink=130:130&_v=20250326163812, 가격: 94040원, 평점: 4.8
수집: Seagate IronWolf Pro 7200/512M, 사진: //img.danawa.com/prod_img/500000/527/645/img/36645527_1.jpg?shrink=130:130&_v=20241206140937, 가격: 982920원, 평점: 4.8
수집: Seagate IronWolf 5400/256M, 사진: //img.danawa.com/prod_img/500000/122/486/img/18486122_1.jpg?shrink=130:130&_v=20250331161749, 가격: 326640원, 평점: 4.5
수집: Western Digital WD Blue 5400/256M, 사진: /

In [5]:
import pandas as pd
df = pd.read_csv("danawa_hdd_products.csv", encoding="UTF-8")
df

,url,Name,Price,Rating,제조회사,등록년월,제품 분류,디스크 크기,디스크 용량,인터페이스,...,5년,데이터 복구 3년,S.M.A.R.T 지원,3년,SED,DSA,저전력,1년,ISE,데이터 복구 2년
0,//img.danawa.com/prod_img/500000/992/764/img/5...,Seagate BarraCuda 5400/256M,199380원,4.7,Seagate,2017년 11월,HDD (PC용),8.9cm(3.5인치),8TB,SATA3(6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,//img.danawa.com/prod_img/500000/078/545/img/6...,Seagate BarraCuda 7200/256M,92190원,4.7,Seagate,2018년 10월,HDD (PC용),8.9cm(3.5인치),2TB,SATA3 (6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,//img.danawa.com/prod_img/500000/460/286/img/4...,Western Digital WD Blue 5640/256M,209900원,4.7,Western Digital WD,2024년 03월,HDD (PC용),8.9cm(3.5인치),8TB,SATA3(6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,//img.danawa.com/prod_img/500000/751/522/img/1...,Western Digital WD Blue 7200/256M,94040원,4.8,Western Digital WD,2021년 03월,HDD (PC용),8.9cm(3.5인치),2TB,SATA3(6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,//img.danawa.com/prod_img/500000/527/645/img/3...,Seagate IronWolf Pro 7200/512M,982920원,4.8,Seagate(제조사 웹사이트 바로가기),2024년 02월,HDD (NAS용),8.9cm(3.5인치),24TB,SATA3 (6Gb/s),...,○,○,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,//img.danawa.com/prod_img/500000/803/335/img/3...,Seagate SpinPoint F3 SATA2/7200/16M/해외구매,200000원,NaN,Seagate,2015년 07월,HDD (PC용),8.9cm(3.5인치),500GB,SATA2(3Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411,//img.danawa.com/prod_img/500000/333/320/img/3...,도시바 MG03ACA 7200/64M/해외구매,160600원,NaN,도시바,2015년 07월,HDD (PC용),8.9cm(3.5인치),3TB,SATA3(6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
412,//img.danawa.com/prod_img/500000/051/958/img/2...,Seagate Surveillance 5900/64M/해외구매,289000원,NaN,Seagate,2014년 06월,HDD (CCTV용),8.9cm(3.5인치),4TB,SATA3(6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
413,//img.danawa.com/prod_img/500000/937/708/img/2...,Seagate Constellation ES.3 7200/128M/해외구매,346770원,NaN,Seagate,2013년 07월,HDD (기업용),8.9cm(3.5인치),4TB,SATA3(6Gb/s),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# c 컬럼의 인덱스 찾기
index = df.columns.get_loc('적합성평가인증')

# c 컬럼 좌측의 컬럼들 선택
left_columns = df.columns[:index]
df = df[left_columns]

# 공백을 기준으로 첫 번째 단어 추출
df['등록년'] = df['등록년월'].str.split().str[0]
df['등록월'] = df['등록년월'].str.split().str[1]
df['제조회사'] = df['제조회사'].str.split('\(').str[0]

df.drop(['등록년월', 'Unnamed: 19'], axis=1, inplace=True)

df

<>:11: SyntaxWarning: invalid escape sequence '\('
<>:11: SyntaxWarning: invalid escape sequence '\('
C:\Users\ms0\AppData\Local\Temp\ipykernel_27012\2674413974.py:11: SyntaxWarning: invalid escape sequence '\('
  df['제조회사'] = df['제조회사'].str.split('\(').str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_27012\2674413974.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['등록년'] = df['등록년월'].str.split().str[0]
C:\Users\ms0\AppData\Local\Temp\ipykernel_27012\2674413974.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

,url,Name,Price,Rating,제조회사,제품 분류,디스크 크기,디스크 용량,인터페이스,회전수,버퍼 용량,전송 속도,기록방식,디스크 수,두께,작업부하량,소음(유휴/탐색),2년,등록년,등록월
0,//img.danawa.com/prod_img/500000/992/764/img/5...,Seagate BarraCuda 5400/256M,199380원,4.7,Seagate,HDD (PC용),8.9cm(3.5인치),8TB,SATA3(6Gb/s),"5,400RPM",메모리 256MB,190MB/s,SMR(PMR),플래터 4장,26.11mm,55 TB/Y,23/25dB,○,2017년,11월
1,//img.danawa.com/prod_img/500000/078/545/img/6...,Seagate BarraCuda 7200/256M,92190원,4.7,Seagate,HDD (PC용),8.9cm(3.5인치),2TB,SATA3 (6Gb/s),"7,200RPM",메모리 256MB,220MB/s,SMR(PMR),플래터 1장,20.2mm,55 TB/Y,NaN,○,2018년,10월
2,//img.danawa.com/prod_img/500000/460/286/img/4...,Western Digital WD Blue 5640/256M,209900원,4.7,Western Digital WD,HDD (PC용),8.9cm(3.5인치),8TB,SATA3(6Gb/s),"5,640RPM",메모리 256MB,215MB/s,CMR(PMR),NaN,26.1mm,NaN,24/28dB,○,2024년,03월
3,//img.danawa.com/prod_img/500000/751/522/img/1...,Western Digital WD Blue 7200/256M,94040원,4.8,Western Digital WD,HDD (PC용),8.9cm(3.5인치),2TB,SATA3(6Gb/s),"7,200RPM",메모리 256MB,215MB/s,SMR(PMR),NaN,26.1mm,NaN,25/27dB,○,2021년,03월
4,//img.danawa.com/prod_img/500000/527/645/img/3...,Seagate IronWolf Pro 7200/512M,982920원,4.8,Seagate,HDD (NAS용),8.9cm(3.5인치),24TB,SATA3 (6Gb/s),"7,200RPM",메모리 512MB,285MB/s,CMR(PMR),NaN,26.11mm,550 TB/Y,NaN,NaN,2024년,02월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,//img.danawa.com/prod_img/500000/803/335/img/3...,Seagate SpinPoint F3 SATA2/7200/16M/해외구매,200000원,NaN,Seagate,HDD (PC용),8.9cm(3.5인치),500GB,SATA2(3Gb/s),"7,200RPM",메모리 16MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015년,07월
411,//img.danawa.com/prod_img/500000/333/320/img/3...,도시바 MG03ACA 7200/64M/해외구매,160600원,NaN,도시바,HDD (PC용),8.9cm(3.5인치),3TB,SATA3(6Gb/s),"7,200RPM",메모리 64MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015년,07월
412,//img.danawa.com/prod_img/500000/051/958/img/2...,Seagate Surveillance 5900/64M/해외구매,289000원,NaN,Seagate,HDD (CCTV용),8.9cm(3.5인치),4TB,SATA3(6Gb/s),"5,900RPM",메모리 64MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014년,06월
413,//img.danawa.com/prod_img/500000/937/708/img/2...,Seagate Constellation ES.3 7200/128M/해외구매,346770원,NaN,Seagate,HDD (기업용),8.9cm(3.5인치),4TB,SATA3(6Gb/s),"7,200RPM",메모리 128MB,NaN,CMR(PMR),NaN,NaN,NaN,NaN,NaN,2013년,07월


In [3]:
df.to_csv("hdd.csv", index=False, encoding="utf-8-sig")

In [3]:
df.head().to_csv("hdd_sample.csv", index=False, encoding="utf-8-sig")

In [4]:
df

,url,Name,Price,Rating,제조회사,제품 분류,디스크 크기,디스크 용량,인터페이스,회전수,버퍼 용량,전송 속도,기록방식,디스크 수,두께,작업부하량,소음(유휴/탐색),2년,등록년,등록월
0,//img.danawa.com/prod_img/500000/992/764/img/5...,Seagate BarraCuda 5400/256M,199380원,4.7,Seagate,HDD (PC용),8.9cm(3.5인치),8TB,SATA3(6Gb/s),"5,400RPM",메모리 256MB,190MB/s,SMR(PMR),플래터 4장,26.11mm,55 TB/Y,23/25dB,○,2017년,11월
1,//img.danawa.com/prod_img/500000/078/545/img/6...,Seagate BarraCuda 7200/256M,92190원,4.7,Seagate,HDD (PC용),8.9cm(3.5인치),2TB,SATA3 (6Gb/s),"7,200RPM",메모리 256MB,220MB/s,SMR(PMR),플래터 1장,20.2mm,55 TB/Y,NaN,○,2018년,10월
2,//img.danawa.com/prod_img/500000/460/286/img/4...,Western Digital WD Blue 5640/256M,209900원,4.7,Western Digital WD,HDD (PC용),8.9cm(3.5인치),8TB,SATA3(6Gb/s),"5,640RPM",메모리 256MB,215MB/s,CMR(PMR),NaN,26.1mm,NaN,24/28dB,○,2024년,03월
3,//img.danawa.com/prod_img/500000/751/522/img/1...,Western Digital WD Blue 7200/256M,94040원,4.8,Western Digital WD,HDD (PC용),8.9cm(3.5인치),2TB,SATA3(6Gb/s),"7,200RPM",메모리 256MB,215MB/s,SMR(PMR),NaN,26.1mm,NaN,25/27dB,○,2021년,03월
4,//img.danawa.com/prod_img/500000/527/645/img/3...,Seagate IronWolf Pro 7200/512M,982920원,4.8,Seagate(제조사 웹사이트 바로가기),HDD (NAS용),8.9cm(3.5인치),24TB,SATA3 (6Gb/s),"7,200RPM",메모리 512MB,285MB/s,CMR(PMR),NaN,26.11mm,550 TB/Y,NaN,NaN,2024년,02월
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,//img.danawa.com/prod_img/500000/803/335/img/3...,Seagate SpinPoint F3 SATA2/7200/16M/해외구매,200000원,NaN,Seagate,HDD (PC용),8.9cm(3.5인치),500GB,SATA2(3Gb/s),"7,200RPM",메모리 16MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015년,07월
411,//img.danawa.com/prod_img/500000/333/320/img/3...,도시바 MG03ACA 7200/64M/해외구매,160600원,NaN,도시바,HDD (PC용),8.9cm(3.5인치),3TB,SATA3(6Gb/s),"7,200RPM",메모리 64MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2015년,07월
412,//img.danawa.com/prod_img/500000/051/958/img/2...,Seagate Surveillance 5900/64M/해외구매,289000원,NaN,Seagate,HDD (CCTV용),8.9cm(3.5인치),4TB,SATA3(6Gb/s),"5,900RPM",메모리 64MB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2014년,06월
413,//img.danawa.com/prod_img/500000/937/708/img/2...,Seagate Constellation ES.3 7200/128M/해외구매,346770원,NaN,Seagate,HDD (기업용),8.9cm(3.5인치),4TB,SATA3(6Gb/s),"7,200RPM",메모리 128MB,NaN,CMR(PMR),NaN,NaN,NaN,NaN,NaN,2013년,07월


In [22]:
# 1. 첫 번째 테이블: idx (1000부터 시작)와 a, b 컬럼
hdd_product_df = pd.DataFrame({
    'product_idx': range(3772, 3772 + len(df)),
    'name': df['Name'],
    'brand': df['제조회사'],
    'price': df['Price'],
    'rating': df['Rating'],
    'category': "HDD"
})

# 2. 두 번째 테이블: idx (index+1), table1의 idx를 외래키로, c 컬럼
hdd_img_df = pd.DataFrame({
    'product_image_idx': range(3772, 3772 + len(df)),
    'product_idx': hdd_product_df['product_idx'],  # table1의 idx를 외래키로
    'img_url': df['url']
})

used_columns = ['Name', 'Price', 'url', 'Rating', '제조회사']
# 원래 데이터프레임에서 사용되지 않은 컬럼 선택
remaining_columns = [col for col in df.columns if col not in used_columns]

# table3 생성
hdd_detail_df = pd.DataFrame({
    'hdd_idx': range(1, len(df) + 1),
    'product_idx': hdd_product_df['product_idx']
})
# 나머지 컬럼 추가
for col in remaining_columns:
    hdd_detail_df[col] = df[col]

In [23]:
hdd_img_df.to_csv("hdd_img.csv", index=False, encoding="utf-8-sig")

In [24]:
hdd_product_df.to_csv("hdd_product.csv", index=False, encoding="utf-8-sig")

In [25]:
hdd_detail_df.to_csv("hdd_detail.csv", index=False, encoding="utf-8-sig")

In [27]:
hdd_detail_df.drop(['디스크 수', '2년'], axis=1, inplace=True)

In [29]:
list(hdd_detail_df)

['hdd_idx',
 'product_idx',
 '제품 분류',
 '디스크 크기',
 '디스크 용량',
 '인터페이스',
 '회전수',
 '버퍼 용량',
 '전송 속도',
 '기록방식',
 '두께',
 '작업부하량',
 '소음(유휴/탐색)',
 '등록년',
 '등록월']